## Model loading and quantization

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device

'cuda'

In [2]:
import gc

gc.collect()
torch.cuda.empty_cache()
gc.collect()

0

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())

True
12.1
90100


In [5]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

model_name = "t-tech/T-lite-it-1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

quantizer = GPTQQuantizer(bits=4, dataset="c4")
quantized_model = quantizer.quantize_model(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

c4-train.00000-of-01024.json.gz:   0%|          | 0.00/319M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (11845 > 8192). Running this sequence through the model will result in indexing errors


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_bmm)

In [8]:
prompt = "Сколько вертолетов может съесть человек?"
messages = [
    {"role": "system", "content": "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Это шутка или вопрос из области мифов и анекдотов. В реальности человек не может съесть вертолет, так как это транспортное средство изготовлено из материалов, которые несъедобны для человека, и его размер значительно превышает человеческие возможности по потреблению пищи. Если у тебя есть другие вопросы или нужна информация, дай знать!


In [9]:
import gc

gc.collect()
torch.cuda.empty_cache()
gc.collect()

0

In [10]:
total_size = sum(param.numel() for param in model.parameters())
print(f"Number of parameters: {total_size / 1000000000:.2f}b")

Number of parameters: 2.72b


## Push quantized model on HF-hub

In [12]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
repo_name = "MilyaShams/T-lite-it-1.0_Q4_0"
model.push_to_hub(repo_name)

In [14]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/milyausha2801/T-lite-it-1.0_Q4_0/commit/111b6820f81834460a4e1d9adaa84b06239fed2c', commit_message='Upload tokenizer', commit_description='', oid='111b6820f81834460a4e1d9adaa84b06239fed2c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/milyausha2801/T-lite-it-1.0_Q4_0', endpoint='https://huggingface.co', repo_type='model', repo_id='milyausha2801/T-lite-it-1.0_Q4_0'), pr_revision=None, pr_num=None)

____

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

tokenizer = AutoTokenizer.from_pretrained('t-tech/T-lite-it-1.0')
gptq_config = GPTQConfig(bits=4, dataset="c4", tokenizer=tokenizer)

In [3]:
qmodel = AutoModelForCausalLM.from_pretrained('t-tech/T-lite-it-1.0',
                                              device_map="auto",
                                              quantization_config=gptq_config)

CUDA extension not installed.
CUDA extension not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


ValueError: disk offload is not supported with GPTQ quantization